In [5]:
#install beautiful soup
!pip install bs4


In [6]:
!pip install lxml

In [7]:
!pip install html5lib

In [8]:
!pip install requests

In [9]:
#packages pour scrapper des pages html
import urllib
import bs4
#help(bs4)

In [10]:
#Etape 1 se connecter à la page allocine et obtenir le code source
url_allo_cine= "http://www.allocine.fr/films/"
from urllib import request

request_text = request.urlopen(url_allo_cine).read()
print(request_text[:5000])

b'<!DOCTYPE html>\n<html>\n<head>\n        <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n        <meta charset="UTF-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no">\n    <meta name="robots" content="index,follow,max-snippet:-1" />\n        <title>Les films les plus populaires - AlloCin\xc3\xa9</title>\n    <meta name="keywords" content="" />\n    <meta name="Googlebot" content="noarchive" />\n    <meta name="distribution" content="global" />\n    <meta name="author" content="AlloCine" />\n    <meta name="country" content="France" />\n    <meta name="geo.position" content="48.87078;2.30447" />\n    <meta name="geo.country" content="FR" />\n            <meta name="ICBM" content="48.87078;2.30447" />\n    <meta name="theme-color" content="#FECC00" />\n    <meta name="description" content="D\xc3\xa9couvrez tous les films les plus populaires sur AlloCin\xc3\xa9, comme :  Une \xc3\xa9tr

In [11]:
#etape 2 : utiliser la packaga beautifulSoup qui "comprend" les balises contenues dans la chaine de caractères renvoyée par la fonction request

page = bs4.BeautifulSoup(request_text,"lxml")

print(page)

<!DOCTYPE html>
<html>
<head>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="index,follow,max-snippet:-1" name="robots"/>
<title>Les films les plus populaires - AlloCiné</title>
<meta content="" name="keywords"/>
<meta content="noarchive" name="Googlebot"/>
<meta content="global" name="distribution"/>
<meta content="AlloCine" name="author"/>
<meta content="France" name="country"/>
<meta content="48.87078;2.30447" name="geo.position"/>
<meta content="FR" name="geo.country"/>
<meta content="48.87078;2.30447" name="ICBM"/>
<meta content="#FECC00" name="theme-color"/>
<meta content="Découvrez tous les films les plus populaires sur AlloCiné, comme :  Une étrange affaire, Le Ruban blanc, Red" name="description"/>
<meta content="AlloCiné" property="og:site_name"/>
<meta content="Les films les plus populaires sur

In [12]:
print(page.find('meta-title'))

None


In [13]:
print(page.find('table'))

None


In [14]:
import requests
from requests import get

In [15]:
#download a page
page = requests.get("http://www.allocine.fr/films/")

In [16]:
page 

<Response [200]>

In [17]:
#print the html content
#page.content

In [18]:
from bs4 import BeautifulSoup

In [19]:
#parse the document
soup = BeautifulSoup(page.content, 'lxml')

In [20]:
#print tje html content of the page formatted nicely
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
  <meta content="index,follow,max-snippet:-1" name="robots"/>
  <title>
   Les films les plus populaires - AlloCiné
  </title>
  <meta content="" name="keywords"/>
  <meta content="noarchive" name="Googlebot"/>
  <meta content="global" name="distribution"/>
  <meta content="AlloCine" name="author"/>
  <meta content="France" name="country"/>
  <meta content="48.87078;2.30447" name="geo.position"/>
  <meta content="FR" name="geo.country"/>
  <meta content="48.87078;2.30447" name="ICBM"/>
  <meta content="#FECC00" name="theme-color"/>
  <meta content="Découvrez tous les films les plus populaires sur AlloCiné, comme :  Une étrange affaire, Le Ruban blanc, Red" name="description"/>
  <meta content="AlloCiné" property="og:site_name"/>
  <meta 

In [21]:
#select the elements at the top level of the page
list(soup.children)

['html',
 <html>
 <head>
 <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
 <meta content="index,follow,max-snippet:-1" name="robots"/>
 <title>Les films les plus populaires - AlloCiné</title>
 <meta content="" name="keywords"/>
 <meta content="noarchive" name="Googlebot"/>
 <meta content="global" name="distribution"/>
 <meta content="AlloCine" name="author"/>
 <meta content="France" name="country"/>
 <meta content="48.87078;2.30447" name="geo.position"/>
 <meta content="FR" name="geo.country"/>
 <meta content="48.87078;2.30447" name="ICBM"/>
 <meta content="#FECC00" name="theme-color"/>
 <meta content="Découvrez tous les films les plus populaires sur AlloCiné, comme :  Une étrange affaire, Le Ruban blanc, Red" name="description"/>
 <meta content="AlloCiné" property="og:site_name"/>
 <meta content="Les films les plus po

In [22]:
[type(item) for item in list(soup.children)]

[bs4.element.Doctype, bs4.element.Tag, bs4.element.NavigableString]

In [23]:
#select the html tag and its children
html = list(soup.children)[1]

In [24]:
list(html.children)

['\n',
 <head>
 <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
 <meta charset="utf-8"/>
 <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
 <meta content="index,follow,max-snippet:-1" name="robots"/>
 <title>Les films les plus populaires - AlloCiné</title>
 <meta content="" name="keywords"/>
 <meta content="noarchive" name="Googlebot"/>
 <meta content="global" name="distribution"/>
 <meta content="AlloCine" name="author"/>
 <meta content="France" name="country"/>
 <meta content="48.87078;2.30447" name="geo.position"/>
 <meta content="FR" name="geo.country"/>
 <meta content="48.87078;2.30447" name="ICBM"/>
 <meta content="#FECC00" name="theme-color"/>
 <meta content="Découvrez tous les films les plus populaires sur AlloCiné, comme :  Une étrange affaire, Le Ruban blanc, Red" name="description"/>
 <meta content="AlloCiné" property="og:site_name"/>
 <meta content="Les films les plus populaires s

In [58]:
 for film in soup.find_all('li',{"class":"mdl"}):
    
    #titre film 
    if film.h2 != None:
        print(film.h2.text)
    else :
        titre_film = 'undefined'
        print(titre_film)
 
    #notes:
    
    if film.find_all('span',{"class":"stareval-note"})!= None:
        notes = film.find_all('span',{"class":"stareval-note"})
        if (len(notes))>1 and (notes != None):
            note_presse = notes[0].text
            note_spect = notes[1].text
        elif len(notes) == 1:
            note_presse = notes[0].text
            note_spect = "undefined"
        else:
            note_presse = "undefined"
            note_spect = "undefined"
            
    else:
        note_presse = "undefined"
        note_spect = "undefined"
    print(note_presse)
    print(note_spect)

    #date
    if film.find('span',{"class":"date"})!= None:
        date = film.find('span',{"class":"date"})
    else:
        date = "undefined"
    print(date.text)
    
    
    #duree
    if film.find('span',{"class":"spacer"}).nextSibling != None:
        duree = film.find('span',{"class":"spacer"}).nextSibling
    else:
        duree = "undefined"
        
    print(duree.strip())
    
    #realisateur
    if film.find('a',{"class":"blue-link"}) != None:
        realisateur = film.find('a',{"class":"blue-link"})
    else:
        realisateur = "undefined"
    print(realisateur.text)
        
    #genre
    genres = film.find_all("div", {"class":"meta-body-item meta-body-info"})
    
    list_of_genres=[]
    for genre in genres:
        if len(genre.find_all("span")) < 4:
            list_of_genres.append(', '.join([span.get_text() for span in genre.find_all("span")[2:]]))
        else:
            list_of_genres.append(', '.join([span.get_text() for span in genre.find_all("span")[3:]]))
    
        print(list_of_genres)    
    
    #resume
    resume = film.find('div', {"class":"content-txt"})
    print(resume.text.strip())


Une étrange affaire

3,6
--
23 décembre 1981
1h 45min
Pierre Granier-Deferre
['Comédie dramatique']
Louis, jeune publicitaire, se voit confier de plus en plus de responsabilités par son nouveau directeur. A tel point que, délaissant sa femme, Nina, elle finit par le quitter.

Le Ruban blanc

4,1
3,7
21 octobre 2009
2h 24min
Michael Haneke
['Drame']
Un village protestant de l'Allemagne du Nord à la veille de la Première Guerre mondiale. D'étranges accidents surviennent et prennent peu à peu le caractère d'un rituel punitif. Qui se cache derrière tout cela ?

Red

2,8
3,5
17 novembre 2010
1h 51min
Robert Schwentke
['Comédie, Action']
L'heure de la retraite a sonné ! Dans certaines professions, la transition peut s'avérer difficile : Franck ne supporte pas l'inactivité, son collègue Joe végète en maison de retraite, Marvin use d'amphétamines et Victoria fait des petits boulots...

Bis

3,2
2,8
18 février 2015
1h 47min
Dominique Farrugia
['Comédie']
Éric et Patrice sont amis depuis le lyc

In [28]:
!pip install pandas

  Using cached pytz-2020.1-py2.py3-none-any.whl (510 kB)


In [29]:
import pandas as pd

In [31]:
# création dataframe
column_names = ["titre_film", "note_presse", "note_spect", "date_sortie", "duree", "réalisateur", "résumé"]
df = pd.DataFrame(columns = column_names)
# df = df.append(df, ignore_index = True)

In [40]:
for film in soup.find_all('li', {"class":"mdl"}):
# genres
    genres=soup.find_all("div", {"class":"meta-body-item meta-body-info"})
    list_of_genres=[]
    for genre in genres:
        if len(genre.find_all("span")) < 4:
            list_of_genres.append(', '.join([span.get_text() for span in genre.find_all("span")[2:]]))
        else:
            list_of_genres.append(', '.join([span.get_text() for span in genre.find_all("span")[3:]]))
    
print(list_of_genres)

['Comédie dramatique', 'Drame', 'Comédie, Action', 'Comédie', 'Comédie, Drame', 'Comédie, Drame', 'Action', 'Romance, Thriller', 'Policier, Action, Drame', 'Fantastique, Thriller', 'Drame', 'Comédie', 'Drame, Science fiction', 'Drame, Comédie, Romance', 'Drame']


In [32]:
for film in soup.find_all('li', {"class":"mdl"}):
    
    # titre film
    if film.h2 != None :
        titre_film = film.h2.text.strip()
    else:
        titre_film = 'undefined'
    print(titre_film)
    
    # notes
    if (film.find_all('span', {"class":"stareval-note"})) != None :
        notes = film.find_all('span', {"class":"stareval-note"})
        if (len(notes) > 1) and (notes != None):
            note_presse = notes[0].text
            note_spect = notes[1].text
        elif len(notes) == 1:
            note_presse = notes[0].text
            note_spect = "undefined"
        else:
            note_presse = "undefined"
            note_spect = "undefined"            
    else:
        note_presse = "undefined"
        note_spect = "undefined"
    print(note_presse)
    print(note_spect)
    # date
    if film.find('span', {"class":"date"}) != None :
        date = date = film.find('span', {"class":"date"}).text
        print(date)
    else:
        date = 'undefined'
        print(date)
    
    # duree
    if film.find('span', {"class":"spacer"}) != None :
        duree = film.find('span', {"class":"spacer"}).nextSibling.strip()
    else:
        duree = "undefined"
    print(duree)
    
    # genre
    if film.find('a', {"class":"xXx"}) != None :
        genre = film.find('a', {"class":"xXx"})
        print(genre)
     # réalisateur
    if film.find('a', {"class":"blue-link"}) != None :
        realisateur = film.find('a', {"class":"blue-link"}).text
    else:
        realisateur = "undefined"
    print(realisateur)
    
    # résumé
    if film.find('div', {"class":"content-txt"}) != None :
        resume = film.find('div', {"class":"content-txt"}).text.strip()
    else:
        resume = "undefined"
    print(resume + '\n')
        
    # append dataframe
    df = df.append({"titre_film":titre_film, 
                    "note_presse":note_presse,
                    "note_spect":note_spect,
                    "date_sortie":date, 
                    "duree":duree, 
                    "réalisateur":realisateur,
                    "résumé":resume}, 
                    ignore_index=True)       

Une étrange affaire
3,6
--
23 décembre 1981
1h 45min
Pierre Granier-Deferre
Louis, jeune publicitaire, se voit confier de plus en plus de responsabilités par son nouveau directeur. A tel point que, délaissant sa femme, Nina, elle finit par le quitter.

Le Ruban blanc
4,1
3,7
21 octobre 2009
2h 24min
Michael Haneke
Un village protestant de l'Allemagne du Nord à la veille de la Première Guerre mondiale. D'étranges accidents surviennent et prennent peu à peu le caractère d'un rituel punitif. Qui se cache derrière tout cela ?

Red
2,8
3,5
17 novembre 2010
1h 51min
Robert Schwentke
L'heure de la retraite a sonné ! Dans certaines professions, la transition peut s'avérer difficile : Franck ne supporte pas l'inactivité, son collègue Joe végète en maison de retraite, Marvin use d'amphétamines et Victoria fait des petits boulots...

Bis
3,2
2,8
18 février 2015
1h 47min
Dominique Farrugia
Éric et Patrice sont amis depuis le lycée. Au fil des années, chacun a pris un chemin très différent : d’un c

In [34]:
df.head(200)

,titre_film,note_presse,note_spect,date_sortie,duree,réalisateur,résumé
0,Une étrange affaire,"3,6",--,23 décembre 1981,1h 45min,Pierre Granier-Deferre,"Louis, jeune publicitaire, se voit confier de ..."
1,Le Ruban blanc,"4,1","3,7",21 octobre 2009,2h 24min,Michael Haneke,Un village protestant de l'Allemagne du Nord à...
2,Red,"2,8","3,5",17 novembre 2010,1h 51min,Robert Schwentke,L'heure de la retraite a sonné ! Dans certaine...
3,Bis,"3,2","2,8",18 février 2015,1h 47min,Dominique Farrugia,Éric et Patrice sont amis depuis le lycée. Au ...
4,A vif !,"2,3","3,0",4 novembre 2015,1h 42min,John Wells,"Plus qu’un grand chef, Adam Jones est une rock..."
5,Le Brio,"3,5","3,9",25 novembre 2016,1h 37min,Yvan Attal,Neïla Salah a grandi à Créteil et rêve de deve...
6,La Chute de Londres,"3,2","2,9",2 mars 2016,1h 39min,Babak Najafi,Les plus grands leaders du monde occidental so...
7,Tenet,undefined,undefined,22 juillet 2020,,Christopher Nolan,Le nouveau film de Christopher Nolan dont l'ac...
8,Le professionnel,"3,6",--,21 octobre 1981,1h 45min,Georges Lautner,"Issu de l'élite de l'armée française, Joss Bea..."
9,The Room,"2,7","3,5",6 mai 2020,1h 40min,Christian Volckman,Kate et Matt quittent la ville pour s'installe...
